In [123]:
import pandas as pd
import requests as r
import re
from bs4 import BeautifulSoup
import time
import numpy as np
import json
import unicodedata
import pymysql
from sqlalchemy import create_engine

## #1 Get the list of materials

I will only work on French market. 

Final structure of url is as follow: `https://www.doctolib.fr/{specialite}/{ville|departement|region|pays}?page={number}`

What I need:
* List of specialties
* List of department, region

In [112]:
# Get the list of specialities
url='https://www.doctolib.fr/specialities'

headers="user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
headers=dict([i.split(': ') for i in headers.split('\n')])

r.get(url,headers=headers)

<Response [200]>

In [113]:
html=r.get(url,headers=headers).content
soup=BeautifulSoup(html)

list_of_specialties=[i["href"].strip('/') for i in soup.select('h1+div a')]
len(list_of_specialties)

123

In [4]:
## Get the list of departments
departments=pd.read_csv('../data/departements-francais.csv',sep="\t") 
departments.head()

,NUMÉRO,NOM,REGION,CHEF LIEU,SUPERFICIE (km²),POPULATION,DENSITE (habitants/km2)
0,1,Ain,Auvergne-Rhône-Alpes,Bourg-en-Bresse,5762.0,631877.0,109.7
1,2,Aisne,Hauts-de-France,Laon,7369.0,538659.0,73.1
2,3,Allier,Auvergne-Rhône-Alpes,Moulins,7340.0,341613.0,46.5
3,4,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,Digne,6925.0,161799.0,23.4
4,5,Hautes-Alpes,Provence-Alpes-Côte d'Azur,Gap,5549.0,140916.0,25.4


In [5]:
index_to_drop=departments.iloc[-2:].index
departments.drop(index_to_drop,axis=0,inplace=True)

In [6]:
departments.columns=departments.columns.map(str.capitalize)

In [7]:
list_of_region=set([str(unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore')).strip("b'").strip('"').lower().replace("'","-").replace(" ","-") for x in departments.loc[:,'Region']])
list_of_region=list(list_of_region)
list_of_region[0]


'grand-est'

In [62]:
list_of_department=[str(unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore')).strip("b'").strip('"').lower().replace("'","-").replace(" ","-") for x in departments.loc[:,'Nom']]
list_of_department[6]


'ardeche'

## #2 List of professionals

In [73]:
# Get the content for one page
base_url='https://www.doctolib.fr'
speciality=list_of_specialties[0]
location='aisne'

final_url=f"{base_url}/{speciality}/{location}?page=9"

headers="""accept: application/json
accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
cache-control: no-cache
content-type: application/json; charset=utf-8
cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
referer: https://www.doctolib.fr/
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
headers=dict([i.split(': ') for i in headers.split('\n')])

# Succeed to reteive a json format for the results of the search
result=r.get(final_url,headers=headers).json()

In [74]:
len(result['data']['doctors'])+len(result['data']['directory_doctors'])

0

In [61]:
data=pd.json_normalize(result['data'])
data[['has_more','is_hub','search_scope','total_search_results']]


,has_more,is_hub,search_scope,total_search_results
0,False,False,speciality,4


In [59]:
data2=pd.json_normalize(result['data']['doctors'])
data2

,id,is_directory,address,city,zipcode,link,cloudinary_public_id,profile_id,exact_match,priority_speciality,first_name,last_name,name_with_title,speciality,organization_status,top_specialities,place_id,telehealth,position.lat,position.lng
0,2473724,False,5 Rue de la Buerie,Soissons,02200,/centre-dentaire/soissons/centre-dentaire-avic...,swp6hdlpxqjrl76xsjet,141882,True,False,None,Centre dentaire Avicenne Soissons,Centre dentaire Avicenne Soissons,None,Centre dentaire,[4 chirurgiens-dentistes],None,False,49.381355,3.324175
1,2021265,False,3 Avenue de la Gare,Villers-Cotterêts,02600,/dentiste/paris/edouard-benchabatt,uqfbutaw5m9ffegchmaw,40395,True,False,Edouard,BENCHABATT,Dr Edouard BENCHABATT,Chirurgien-dentiste,None,NaN,practice-96074,False,49.250568,3.093963
2,2306416,False,50 Rue de Vervins,Hirson,02500,/dentiste/hirson/jean-francois-seret,be5tsaipx2rhvwjgi7ki,29712,True,False,Jean-François,SERET,Dr Jean-François SERET,Chirurgien-dentiste,None,NaN,None,False,49.921050,4.082104
3,2210606,False,3 Avenue de la Gare,Villers-Cotterêts,02600,/dentiste/asnieres-sur-seine/samson-elbez,apu8z0a56y2tiz9uhwq2,147357,True,False,Samson,Elbez,Dr Samson Elbez,Chirurgien-dentiste,None,NaN,practice-96074,False,49.250568,3.093963
4,2152950,False,26 Rue de Reims,Sissonne,02150,/dentiste/sissonne/anne-charlotte-robin,etrltv6xdztlb9x1gqkf,121202,True,False,Anne-Charlotte,ROBIN,Dr Anne-Charlotte ROBIN,Chirurgien-dentiste,None,NaN,None,False,49.566025,3.892713
5,2152962,False,26 Rue de Reims,Sissonne,02150,/dentiste/sissonne/emmanuel-robin,cinijxsaub2bq2ixinxc,121203,True,False,Emmanuel,ROBIN,Dr Emmanuel ROBIN,Chirurgien-dentiste,None,NaN,None,False,49.566025,3.892713
6,2301939,False,2 Rue de la 3ème Division d'Infanterie Motorisee,Saint-Quentin,02100,/dentiste/saint-quentin/samuel-seube,r5byqlwavlfibozjjhtz,27168,True,False,Samuel,SEUBE,Dr Samuel SEUBE,Chirurgien-dentiste,None,NaN,None,True,49.848688,3.277489
7,2152982,False,26 Rue de Reims,Sissonne,02150,/dentiste/sissonne/estelle-boudereaux,otuwi3truke37sagnh7i,121205,True,False,Estelle,BOUDEREAUX,Dr Estelle BOUDEREAUX,Chirurgien-dentiste,None,NaN,None,False,49.566025,3.892713
8,2354973,False,6 Résidence Rameau,Château-Thierry,02400,/dentiste/chateau-thierry/nicolas-fortin,zyx08rvgp78w2pg4itg9,76031,True,False,Nicolas,FORTIN,Dr Nicolas FORTIN,Chirurgien-dentiste,None,NaN,None,False,49.061658,3.392244
9,2247602,False,29 Rue Jean Baptiste Lebas,Laon,02000,/dentiste/laon/silvia-geambasu,qgntapsurfn2na1fnt3g,116317,True,False,Silvia,GEAMBASU,Dr Silvia GEAMBASU,Chirurgien-dentiste,None,NaN,None,False,49.568750,3.613438


In [96]:
# Get the content with all pages
base_url='https://www.doctolib.fr'
speciality=list_of_specialties[0]
location=list_of_department[6]

df_final=pd.DataFrame()
i=0
doctors=True

while doctors:
    
    i+=1
    
    final_url=f"{base_url}/{speciality}/{location}?page={i}"
    print("final_url: ",final_url)
    
    headers=f"""accept: application/json
    accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
    cache-control: no-cache
    content-type: application/json; charset=utf-8
    cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
    referer: https://www.doctolib.fr/{speciality}/{location}?page={1 if i<2 else i-1}
    user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
    x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
    headers=dict([i.strip().split(': ') for i in headers.split('\n')])
    
    result=r.get(final_url,headers=headers).json()
    #variable doctors allows to set the while loop to False when there is no more results
    doctors=(len(result['data']['doctors'])!=0) or (len(result['data']['directory_doctors'])!=0)
    
    if len(result['data']['doctors'])>0:
        data=pd.json_normalize(result['data']['doctors'])
        data['doctolib_profile']=True
        df_final=df_final.append(data)
    
    if len(result['data']['directory_doctors'])>0:
        data=pd.json_normalize(result['data']['directory_doctors'])
        data=data.rename(columns={'zip_code':'zipcode'})
        data['doctolib_profile']=False
        data['profile_id']=-1
        df_final=df_final.append(data)
            
    time.sleep(1)
    

final_url:  https://www.doctolib.fr/dentiste/ardeche?page=1
result['data']['doctors'] 1
result['data']['directory_doctors'] 30
final_url:  https://www.doctolib.fr/dentiste/ardeche?page=2
result['data']['doctors'] 0
result['data']['directory_doctors'] 30
final_url:  https://www.doctolib.fr/dentiste/ardeche?page=3
result['data']['doctors'] 0
result['data']['directory_doctors'] 30
final_url:  https://www.doctolib.fr/dentiste/ardeche?page=4
result['data']['doctors'] 0
result['data']['directory_doctors'] 30
final_url:  https://www.doctolib.fr/dentiste/ardeche?page=5
result['data']['doctors'] 0
result['data']['directory_doctors'] 30
final_url:  https://www.doctolib.fr/dentiste/ardeche?page=6
result['data']['doctors'] 0
result['data']['directory_doctors'] 30
final_url:  https://www.doctolib.fr/dentiste/ardeche?page=7
result['data']['doctors'] 0
result['data']['directory_doctors'] 30
final_url:  https://www.doctolib.fr/dentiste/ardeche?page=8
result['data']['doctors'] 0
result['data']['directo

In [100]:
df_final.shape

(229, 20)

In [ ]:
df_final.columns
# drop columns: 'id', 'is_directory','cloudinary_public_id','exact_match', 'priority_speciality'

In [99]:
df_final.head()

,id,is_directory,address,city,zipcode,link,cloudinary_public_id,profile_id,exact_match,priority_speciality,first_name,last_name,name_with_title,speciality,organization_status,place_id,telehealth,position.lat,position.lng,doctolib_profile
0,2467479,False,Rue du Cinéma,Chomerac,07210,/dentiste/marseille/eric-bonnier,tjokotd6zjejo2p5ikwi,139514,True,False,Eric,BONNIER,Dr Eric BONNIER,Chirurgien-dentiste,None,None,False,44.708595,4.661693,True
0,640358,True,7 Rue Marcel Nicolas,Le Pouzin,07250,/dentiste/le-pouzin/jacques-duband,NaN,-1,NaN,NaN,Jacques,DUBAND,Dr Jacques DUBAND,Chirurgien-dentiste,NaN,NaN,NaN,44.755852,4.748168,False
1,110658,True,Maison Medicale 6 Place Du General De Gaulle,Ruoms,07120,/dentiste/ruoms/thomas-reimlinger,NaN,-1,NaN,NaN,Thomas,REIMLINGER,Dr Thomas REIMLINGER,Chirurgien-dentiste,NaN,NaN,NaN,44.454012,4.344194,False
2,899761,True,14 Avenue De Bellande (CENTRE HOSPITALIER D'AU...,Aubenas,07200,/dentiste/aubenas/sandrine-kazimierski-aubenas,NaN,-1,NaN,NaN,Sandrine,Kazimierski,Dr Sandrine Kazimierski,Chirurgien-dentiste,NaN,NaN,NaN,44.613120,4.398460,False
3,112143,True,Le Mercure 426 Avenue Georges Clemenceau,Guilherand Granges,07500,/dentiste/guilherand-granges/michel-mienville,NaN,-1,NaN,NaN,Michel,MIENVILLE,Dr Michel MIENVILLE,Chirurgien-dentiste,NaN,NaN,NaN,44.932214,4.872737,False


In [ ]:
# Final scrapping for all specialities
base_url='https://www.doctolib.fr'
location='france'

df_final=pd.DataFrame()

for i in range(len(list_of_specialties)):
    
    speciality=list_of_specialties[i]
    i=0
    doctors=True
    
    while doctors:
    
    i+=1
    
    final_url=f"{base_url}/{speciality}/{location}?page={i}"
    print("final_url: ",final_url)
    
    headers=f"""accept: application/json
    accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
    cache-control: no-cache
    content-type: application/json; charset=utf-8
    cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJD7fiBh9u5kcRYJSMaMOCCwQ%22%2C%22name%22%3A%22Paris%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTEwVDExOjMwOjQ5LjM3MloiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--34c8fb3207d661d5f08f5fd3388cb219e6a450ca; _doctolib_session=Hhv7pWEDJwdnyCQyqr7ThfPEQcS62D9lKg7M%2Fvz8R0AIHipjT1Hu8SN0rhu51Urw8FXsHuPmX0L2UfmzU62ZifdY0%2Ful6siBegn1tBS9xIOE68s0L8WvzGo2NpZ3WtP8lg7du3f9lA%2BDUY9qvMjbXWQbsnHeE2GdX%2F46rLZZ%2FDvPRwaKV4ZcEYrliAJoVx%2BTeyLg634T5KloWv2A4kStpH7OzTg2A%2FQs8TpEaffKhJckYWtV7uE0BQYcDW14sU3N18VFz1X3RU3uXoAhQxw7%2FcQgncDD4sdPdsO%2FUY7ou2T0IOMmi%2FkJL%2B2sVlpGRgfv21I5BaJFK0fu9l3YgjDL5BuvupSKtAxlaw%3D%3D--jN4XvOBX1kwEvjNq--YuCqF8thsE1OjQaZOd%2B%2Fug%3D%3D
    referer: https://www.doctolib.fr/{speciality}/{location}?page={1 if i<2 else i-1}
    user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
    x-csrf-token: +AD1NNffpu2TUGR+i552cv/SJOdpo7KzgJOwVBys6r3/EBA3AwlvUidQ0R+FVeD3cUpNqeH+LYB2447FTjv7Rg=="""
    headers=dict([i.strip().split(': ') for i in headers.split('\n')])
    
    result=r.get(final_url,headers=headers).json()
    #variable doctors allows to set the while loop to False when there is no more results
    doctors=(len(result['data']['doctors'])!=0) or (len(result['data']['directory_doctors'])!=0)
    
    if len(result['data']['doctors'])>0:
        data=pd.json_normalize(result['data']['doctors'])
        data['doctolib_profile']=True
        df_final=df_final.append(data)
    
    if len(result['data']['directory_doctors'])>0:
        data=pd.json_normalize(result['data']['directory_doctors'])
        data=data.rename(columns={'zip_code':'zipcode'})
        data['doctolib_profile']=False
        data['profile_id']=-1
        df_final=df_final.append(data)
            
    time.sleep(1)

In [101]:
# Save the data in a database
username='root'
host='localhost'
database='doctolib_db'
password=''

engine=create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

In [102]:
# reset index to have a unique id for each row
df_final=df_final.reset_index()

In [106]:
# drop useless columns before saving the data
col_to_drop=['id','index','is_directory','cloudinary_public_id','exact_match']
df_final=df_final.drop(col_to_drop,axis=1)

KeyError: "['id' 'index' 'cloudinary_public_id' 'exact_match'] not found in axis"

In [105]:
# send data into the database
df_final.to_sql('doctors',engine,if_exists='replace',index=False)

## #3 Get availability

Workflow:
* Ask to the user for a list of doctors
* Get the list from the database created from previous part
* Get the enriched information for each profile
* Access to the availability data thanks to enriched information

Structure of the availability url: `https://www.doctolib.fr/availabilities.json?start_date=2020-04-10&visit_motive_ids=1235300&agenda_ids=202675&limit=4`

In [136]:
link='dentiste/asnieres-sur-seine/samson-elbez'
doctor_slug=link[(re.search('(.*/){2}',link).end()):]
doctor_slug

'samson-elbez'

In [138]:
url=f'https://www.doctolib.fr/booking/{doctor_slug}.json'

headers=f"""accept: application/json
accept-language: fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7
content-type: application/json; charset=utf-8
cookie: __cfduid=d2b5aa87f9a6624d7b6d080621c78dd071584958668; ssid=c8003987149mac-5zfwbZlNIgv0; cookie_consent=true; esid=Hag6WnEswqPNmOsSdO6nq8Jg; last_place=%7B%22place_id%22%3A%22ChIJ29CbGL9S6EcRkCVhgT7xCgM%22%2C%22name%22%3A%22Aisne%22%7D; utm_b2b=eyJfcmFpbHMiOnsibWVzc2FnZSI6IkJBaEpJaUoxZEcxZmMyOTFjbU5sUFdScGNtVmpkQ1oxZEcxZmJXVmthWFZ0UFFZNkJrVlUiLCJleHAiOiIyMDIwLTA0LTExVDE0OjQ5OjE3LjQ0M1oiLCJwdXIiOiJjb29raWUudXRtX2IyYiJ9fQ%3D%3D--86af30e089255e9939a367ff59042ed80385d9df; _doctolib_session=OJbFmCwXUFFExx3D7ArhPZAcTmY%2FiuV45MFME%2BOWOqZHxHN2fm33buQ4ZIXhJYZC34rS6Dkbj9faL3khcXcZCLxEqkNxi8BJMzNzmdRcnLkl0s7NvJC%2FFrsRBzXugfb6GQ%2FsNcOB8feIqEd0ZzbVUahP0UN%2FDbh1UVdODs6E46xVDBNkLSs%2BkATECtQpeB%2BC0dp373gQBoHeYovxDVk2bixGyapLNiucG3k7d5a130qPFJBmnThKAI4zW7No%2FyqQEKzKPjLeuVtyVcxah18v2UeIw2pa%2Bols0KF6Grt2TOOKBxgB5yBqhWHYQbDUq7jx6wMYDnnGZDQwCfWWmB%2FqikKsm9A3lhMCA2%2Bkk6uWL6K6%2F43Wzj9WmuvfHUH8XmyaAUJKwav7vmAnH0Ze3qnuXFdTmiR4lEs93k2nIEfKMcOz6V6Z%2FgMo23w%3D--a4gEopZ%2BrN8qgCQC--SboWN%2F0tvvy8vPqtrWicEA%3D%3D
referer: https://www.doctolib.fr/{link}
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36
x-csrf-token: HPo1YOpT3IX4imTZ1vYTpcdC3kXw36ciB1dwvSxUZH8b6tBjPoUVOkyK0bjYPYUgSdq3C3iCOBHxJ04sfsN1hA=="""
headers=dict([i.strip().split(': ') for i in headers.split('\n')])

result=r.get(url,headers=headers).json()


In [ ]:
data_request=dict()

In [140]:
# Get the first visit_motives_id
visit_motives=pd.json_normalize(result['data']['visit_motives'][0])
visit_motives['id']

0    1452879
Name: id, dtype: int64

In [143]:
# Get the first agenda_id
agendas=pd.json_normalize(result['data']['agendas'][0])
agendas['id']

0    239605
Name: id, dtype: int64

In [142]:
data_request

,specialities,visit_motive_categories,visit_motives,agendas,places,practitioners,profile.organization,profile.id,profile.name_with_title_and_determiner
0,"[{'id': 1, 'name': 'Chirurgien-dentiste', 'kin...",[],"[{'id': 1452879, 'name': 'Soins dentaires', 'v...","[{'id': 239605, 'booking_disabled': False, 'bo...","[{'id': 'practice-68775', 'address': '15 Rue P...","[{'id': 16905611, 'profile_id': 147357, 'cloud...",False,147357,le Dr Samson Elbez
